# COCO based object classification

In [ ]:
from util.PreProcessing import filterDataset, dataGeneratorCoco
from util.getModel import build_model
from tensorflow.keras import models, losses

## For visualizing results
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
%matplotlib inline

## Importing the Coco dataset, filtering and creating generator objects.

In [5]:
# Preprocessing arguments
folder = 'D:/COCOdata'
classes = ['cat', 'dog', 'horse']
batch_size = 4
input_image_size = (224,224)
mask_type = 'normal'
augGeneratorArgs = dict(featurewise_center = False, 
                        samplewise_center = False,
                        rotation_range = 5, 
                        width_shift_range = 0.01, 
                        height_shift_range = 0.01, 
                        brightness_range = (0.8,1.2),
                        shear_range = 0.01,
                        zoom_range = [1, 1.25],  
                        horizontal_flip = True, 
                        vertical_flip = False,
                        fill_mode = 'reflect',
                        data_format = 'channels_last')

# -> Create filtered train dataset (using filterDataset())
images_t, labels_t, dataset_size_t, coco_t = filterDataset(folder, classes,  mode='train2017')
# -> Create train generator (using dataGeneratorCoco())
train_gen = dataGeneratorCoco(images_t, labels_t, coco_t, folder,
                              input_image_size, batch_size, mode="train2017")
#train_gen_aug = augmentationsGenerator(train_gen, augGeneratorArgs)


# -> Create filtered val dataset (using filterDataset()) 
images_v, labels_v, dataset_size_v, coco_v = filterDataset(folder, classes,  mode='val2017')
# -> Create val generator (using dataGeneratorCoco()) 
mode="val2017";
val_gen = dataGeneratorCoco(images_v, labels_v, coco_v, folder,
                            input_image_size, batch_size, mode)
# val_gen_aug = augmentationsGenerator(val_gen, augGeneratorArgs) # <-- fix this later

loading annotations into memory...
Done (t=21.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.69s)
creating index...
index created!


## Building a model, training and evaluating it.

In [19]:
inputshape = input_image_size + tuple([3]) # three colours means three dimentions
nr_of_classes = len(classes)

model = build_model(inputshape, nr_of_classes)

model.summary()

(224, 224, 3)
Model: "VGG19_U-Net"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 5

In [16]:
opt = 'adam'
lossFn = losses.CategoricalCrossentropy(from_logits=True) 

# Compile your model first
model.compile(loss = lossFn, optimizer = opt, metrics=['accuracy'])

In [21]:
# Start the training process
history = model.fit(x = train_gen,
                validation_data = val_gen,
                steps_per_epoch = steps_per_epoch,
                validation_steps = validation_steps,
                epochs = n_epochs,
                verbose = True)

Epoch 1/4
2779/2779 [==============================] - 3344s 1s/step - loss: 1.6099 - accuracy: 0.6250 - val_loss: 1.2113 - val_accuracy: 0.6949
Epoch 2/4
2779/2779 [==============================] - 2970s 1s/step - loss: 1.2370 - accuracy: 0.7147 - val_loss: 1.8945 - val_accuracy: 0.5953
Epoch 3/4
2779/2779 [==============================] - 2873s 1s/step - loss: 0.9524 - accuracy: 0.7687 - val_loss: 2.4054 - val_accuracy: 0.6568
Epoch 4/4
2779/2779 [==============================] - 2857s 1s/step - loss: 0.8635 - accuracy: 0.7970 - val_loss: 1.8489 - val_accuracy: 0.6525


In [23]:
# Save the model architecture, weights, optimizer and loss metrics
model.save("models/VGG19-Imagenet-14122021")

INFO:tensorflow:Assets written to: models/VGG19-Imagenet-14122021\assets


#### Tip:
You can create a separate .py file (say cocoFunctions.py) and copy the code for the imports, filterDataset function, and dataGeneratorCoco + supporting functions. This will help make the code more systematic.

You can then simply import the functions into any code by using:  
from cocoFunctions import filterDataset, dataGeneratorCoco